Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 85.5 MB/s eta 0:00:00


Cirq (durum vektörü ile tam beklenen değer)

In [2]:
import numpy as np
import cirq

# durum
psi = np.array([1/np.sqrt(3), np.sqrt(2/3)], dtype=complex)

# Pauli-X matrisi
X = np.array([[0,1],[1,0]], dtype=complex)

# beklenen değer
expval = np.vdot(psi, X @ psi)   # <psi| X |psi>
print("Cirq (statevector) ⟨X⟩ =", expval.real)  # gerçek kısmı yazdır
# Beklenen: 2*sqrt(2)/3 ≈ 0.942809


Cirq (statevector) ⟨X⟩ = 0.9428090415820635


Qiskit (statevector ile analitik; örneklemeli simülasyon da var)

In [5]:
import numpy as np
from qiskit.quantum_info import Statevector, Operator
from qiskit import QuantumCircuit, Aer, transpile

# durum vektörü
psi = Statevector([1/np.sqrt(3), np.sqrt(2/3)])

# X operatörü
X = Operator([[0,1],[1,0]])

# analitik beklenen değer
expval = np.real(psi.data.conj().T @ (X.data @ psi.data))
print("Qiskit (statevector) ⟨X⟩ =", expval)

# --- yaklaşık ölçümle doğrulama (shot-based) ---
qc = QuantumCircuit(1,1)
# hazır durumu hazırlamak: genel amplitüd hazırlama (u3 yerine direk initialize)
qc.initialize(psi.data, 0)
# X ölçümü için H -> Z ölçümü (X = H Z H)
qc.h(0)
qc.measure(0,0)
sim = Aer.get_backend('aer_simulator')
tqc = transpile(qc, sim)
qobj = sim.run(tqc, shots=5000).result()
counts = qobj.get_counts()
# hesaplanan beklenen değer: (+1)*(p0) + (-1)*(p1)
p0 = counts.get('0',0)/5000
p1 = counts.get('1',0)/5000
approx_exp = p0 - p1
print("Qiskit (sampling approx) ⟨X⟩ ≈", approx_exp)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QuTiP (analitik ve yoğunluk matrisi ile)

In [6]:
import numpy as np
from qutip import Qobj

# durum
psi = Qobj([1/np.sqrt(3), np.sqrt(2/3)])   # column vector

# Pauli-X
X = Qobj([[0,1],[1,0]])

# analitik beklenen değer
expval = (psi.dag() * X * psi)[0,0]
print("QuTiP ⟨X⟩ =", np.real(expval))

# yoğunluk matrisi ile de aynı sonucu alırız
rho = psi * psi.dag()
print("Tr(rho X) =", np.real((rho * X).tr()))


TypeError: 'complex' object is not subscriptable